In [28]:
import data_provider
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



In [29]:
class Net(nn.Module):

    def __init__(self, input_size, hidden_size, output_size, activation_fn = 'relu', apply_dropout=False):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

        self.hidden_size = hidden_size
        self.activation_fn = activation_fn

        self.dropout = None
        if apply_dropout:
            self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        activation_fn = None
        if self.activation_fn == 'sigmoid':
            activation_fn = F.sigmoid
        elif self.activation_fn == 'tanh':
            activation_fn = torch.tanh
        elif self.activation_fn == 'relu':
            activation_fn = F.relu

        x = activation_fn(self.fc1(x))
        x = activation_fn(self.fc2(x))
        if self.dropout is not None :
            x = self.dropout(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=-1)

def train_and_evaluate_model(
        model,
        Xtrain, Ytrain,
        Xtest, Ytest,
        learn_rate=0.001
):
    epoch_data = []
    epochs = 1001

    optimizer = optim.Adam(model.parameters(), lr=learn_rate)
    loss_fn = nn.NLLLoss()

    test_accuracy = 0.0
    for epoch in range(1, epochs):
        optimizer.zero_grad()

        model.train()
        Ypred = model(Xtrain)
        loss = loss_fn(Ypred, Ytrain)
        loss.backward()

        optimizer.step()

        model.eval()
        Ypred_test = model(Xtest)
        loss_test = loss_fn(Ypred_test, Ytest)

        _, pred = Ypred_test.data.max(1)

        if epoch % 100 == 0:
            print('epoch - {} train loss - {} test loss - {} Test Accuracy - {}'.format(
                epoch, loss.data.item(), loss_test.data.item(), test_accuracy
            ))

for dataset in data_provider.data():
    features_train = dataset['train'][0]
    labels_train = dataset['train'][1]

    features_test = dataset['test'][0]
    labels_test = dataset['test'][1]

    model = Net(input_size=features_train.shape[1], hidden_size=100, output_size=2)

    train_and_evaluate_model(
        model,
        torch.tensor(features_train.values, dtype=torch.float), torch.tensor(labels_train),
        torch.tensor(features_test.values, dtype=torch.float), torch.tensor(labels_test)
    )

epoch - 100 train loss - 0.584395706653595 test loss - 0.620108962059021 Test Accuracy - 0.0
epoch - 200 train loss - 0.49668896198272705 test loss - 0.5586620569229126 Test Accuracy - 0.0
epoch - 300 train loss - 0.4706190526485443 test loss - 0.5443929433822632 Test Accuracy - 0.0
epoch - 400 train loss - 0.4534396231174469 test loss - 0.5367163419723511 Test Accuracy - 0.0
epoch - 500 train loss - 0.44069328904151917 test loss - 0.5375409722328186 Test Accuracy - 0.0
epoch - 600 train loss - 0.42980101704597473 test loss - 0.5353983640670776 Test Accuracy - 0.0
epoch - 700 train loss - 0.42226845026016235 test loss - 0.5380111336708069 Test Accuracy - 0.0
epoch - 800 train loss - 0.4115374982357025 test loss - 0.541277289390564 Test Accuracy - 0.0
epoch - 900 train loss - 0.40614864230155945 test loss - 0.5554990768432617 Test Accuracy - 0.0
epoch - 1000 train loss - 0.39624762535095215 test loss - 0.5560532808303833 Test Accuracy - 0.0
epoch - 100 train loss - 0.28315722942352295 t